In [ ]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from comstock_processor import ComStockProcessor

dataset_path = Path().resolve() / "datasets"
print(f"Dataset path: {dataset_path}")

comstock_path = dataset_path / "comstock"
print(f"ComStock path: {comstock_path}")

espm_path = dataset_path / "espm"
print(f"ESPM path: {espm_path}")

figures_path = Path().resolve() / "figures"
print(f"Figures path: {figures_path}")

# auto reload any changes in the module
%load_ext autoreload

warnings.filterwarnings("ignore", category=FutureWarning)
# ignore SettingWithCopyWarning
pd.options.mode.chained_assignment = None

# ComStock

### Pull down ComStock data


In [ ]:
# Download and convert the data file to a dataframe.
processor = ComStockProcessor(state="All", county_name="All", building_type="All", upgrade="0", base_dir=comstock_path)
df_all = processor.process_metadata(save_dir=comstock_path)
display(df_all)

### Read in ComStock data

In [ ]:
# get the dimensions
print(df_all.shape)
# show all states
print(df_all["in.state"].unique())
# show all the building types
print(df_all["in.comstock_building_type"].unique())

In [ ]:
# save the names of the fields to a list
fields = df_all.columns
with open(comstock_path / "fields.txt", "w") as f:
    f.write("\n".join(fields))

# force the climate zone to be a string
df_all["in.ashrae_iecc_climate_zone_2006"] = df_all["in.ashrae_iecc_climate_zone_2006"].astype(str)
# print the unique values
print(df_all["in.ashrae_iecc_climate_zone_2006"].unique())

In [ ]:
# calculate some new columns
df_all["EUI_kBTUft2"] = df_all["out.site_energy.total.energy_consumption_intensity"] * 3.412  # kwh/ft2 to kbtu/ft2

# find if climate zone 7A or 7B or 7
df_cz7 = df_all[df_all["in.ashrae_iecc_climate_zone_2006"].str.contains("7")]
print(df_cz7["in.state"].unique())
df_co_cz7 = df_cz7[df_cz7["in.state"] == "CO"]
df_pitkin = df_all[df_all["in.county_name"] == "CO, Pitkin County"]

# print the dimensions for each dataframe
print(f"all: {df_all.shape}")
print(f"cz7: {df_cz7.shape}")
print(f"cz7 co: {df_co_cz7.shape}")
print(f"cz7 pitkin: {df_pitkin.shape}")

In [ ]:
display(df_cz7)
display(df_pitkin)

# # list all the columns that have "weighted" and "intensity" in the name
# weighted_cols = [col for col in df_all.columns if 'weighted' in col]
# print(weighted_cols)

In [ ]:
# use seaborn

# return the quartiles of the column out.site_energy.total.energy_consumption_intensity
print(df_cz7["EUI_kBTUft2"].describe())

# create density plot
sns.kdeplot(df_cz7["EUI_kBTUft2"], shade=True)
# add dashed lines for the quartiles
describe_data = df_cz7["EUI_kBTUft2"].describe()
plt.axvline(describe_data["25%"], color="r", linestyle="--")
plt.axvline(describe_data["50%"], color="g", linestyle="--")
plt.axvline(describe_data["75%"], color="b", linestyle="--")
plt.xlabel("Site Energy EUI (kBtu/ft2)")
plt.title(f"All Building Types - n={len(df_cz7)}")
# display the quartiles in a text box
plt.text(0.5, 0.5, describe_data, transform=plt.gca().transAxes)
plt.xlim(0)

# add in the df_piitkin data
sns.kdeplot(df_pitkin["EUI_kBTUft2"], shade=True)
plt.show()

In [ ]:
# create KDE's for only "SmallOffice", "MediumOffice", "LargeOffice"
df_cz7_office = df_cz7[df_cz7["in.comstock_building_type"].isin(["SmallOffice", "MediumOffice", "LargeOffice"])]
df_cz7_co_office = df_co_cz7[df_co_cz7["in.comstock_building_type"].isin(["SmallOffice", "MediumOffice", "LargeOffice"])]

# create kernel density plot of the column EUI_kBTUft2, using seaborn
sns.kdeplot(df_cz7_office["EUI_kBTUft2"], fill=True, legend=True)
# add in the df_piitkin data
sns.kdeplot(df_cz7["EUI_kBTUft2"], fill=True, legend=True)
sns.kdeplot(df_cz7_co_office["EUI_kBTUft2"], fill=True, legend=True)
# add dashed lines for the quartiles
describe_data = df_cz7_office["EUI_kBTUft2"].quantile([0, 0.25, 0.5, 0.75, 1.0])
describe_data = describe_data.to_dict()
display(describe_data)
# show only describe from the column EUI_kBTUft2

# add line at 47.2 called CBECS C/VC CZ Office
plt.axvline(47.2, color="k", linestyle="--", label="CBECS C/VC CZ Office")
plt.axvline(60, color="k", linestyle=":", label="ASHRAE Std 100")

plt.axvline(describe_data[0.25], color="r", linestyle="--")
plt.axvline(describe_data[0.5], color="g", linestyle="--")
plt.axvline(describe_data[0.75], color="b", linestyle="--")
plt.xlabel("Site Energy EUI - Offices (kBtu/ft2)")
plt.title("Distribution of Office Building Types EUI")


# display the quartiles in a text box
textstr = "ComStock All CZ7 Offices Stats:\n" + "\n".join([f"{k*100}%: {v:.2f}" for k, v in describe_data.items()])

plt.text(0.5, 0.1, textstr, transform=plt.gca().transAxes)
plt.xlim(0, 350)

# add legend
plt.legend(["CZ7 Offices (All)", "CZ7 Buildings (All)", "CZ7 Offices (in CO)", "CBECS C/VC CZ Office 25%-ile", "Std100 EUI Target 25%-ile"])

plt.show()

# CBECS



In [ ]:
# read in the xlsx spreadsheet sheet called cbecs2018_final_public

cbecs2018 = pd.read_excel(dataset_path / "cbecs" / "cbecs2018_final_public.xlsx", sheet_name="cbecs2018_final_public")

In [ ]:
display(cbecs2018)

In [ ]:
# calculate the total kbtu/ft2 for each building
cbecs2018["total_kbtu_ft2"] = cbecs2018["MFBTU"] / cbecs2018["SQFT"]

# create the kde plot of all kbtu/ft2
sns.kdeplot(cbecs2018["total_kbtu_ft2"], fill=True)

# filter only offices
offices = cbecs2018[cbecs2018["PBA"] == 2]
sns.kdeplot(offices["total_kbtu_ft2"], fill=True)

# only offices in very cold
offices_very_cold = offices[offices["PUBCLIM"] == 1]
sns.kdeplot(offices_very_cold["total_kbtu_ft2"], fill=True)
# add legend
plt.legend(["All", "Offices", "Offices in Very Cold"])
plt.xlim(0, 200)
plt.title("CBECS Offices")


# 1st quartile of the offices_very_cold
offices_very_cold_1st_quartile = offices_very_cold["total_kbtu_ft2"].quantile(0.25)
# plot as line
plt.axvline(offices_very_cold_1st_quartile, color="black", linestyle=":")

# ESPM Colorado Site EUI
https://portfoliomanager.energystar.gov/dataExplorer/?_gl=1*1cjut0r*_ga*NzE4OTQ5MDM4LjE3MjM1ODY2NzY.*_ga_S0KJTVVLQ6*MTcyMzU4NjY3Ni4xLjAuMTcyMzU4NjY3Ni4wLjAuMA..


In [ ]:
# show histogram of the building types - in.comstock_building_type

# It would be nice to have this image for each of the data sources
sns.histplot(df_cz7["in.comstock_building_type"])
plt.xlabel("Building Type")
plt.title("Building Types in All Climate Zone 7")
plt.xticks(rotation=90)
plt.show()

# show histogram of the building types - in.comstock_building_type
sns.histplot(df_co_cz7["in.comstock_building_type"])
plt.xlabel("Building Type")
plt.title("Building Types in CO Climate Zone 7")
plt.xticks(rotation=90)
plt.show()

# Data Comparison

In [ ]:
# Read in std data
std100_25th_percentile = pd.read_excel(dataset_path / "std100" / "std100data.xlsx", sheet_name="site_energy_25th_percentile")
std100_40th_percentile = pd.read_excel(dataset_path / "std100" / "std100data.xlsx", sheet_name="site_energy_40th_percentile")
std100_50th_percentile = pd.read_excel(dataset_path / "std100" / "std100data.xlsx", sheet_name="site_energy_50th_percentile")

# display(std25thPercentile)

# Read in ESPM data
espm_data = pd.read_excel(dataset_path / "espm" / "ESPM_AllBuildingTypes_AllClimateZones_AllPercentiles.xlsx", sheet_name="Report")

# display(espm_data)

In [ ]:
def all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
):
    # CBECS Data Handling:
    #   calculate the total kbtu/ft2 for each building
    #   add in literal column for data_source for union and display in visualization
    #   Filter to climate zone (See lookups.py or climate_zone_crosswalk.xlsx in the cbecs data folder for additional definitions for different numbers, not the same as climate zone numbers)
    #   Select data needed for union and visualization
    #   Note if needed: PBA is building type
    cbecs2018["EUI_kBTUft2"] = cbecs2018["MFBTU"] / cbecs2018["SQFT"]
    cbecs2018["data_source"] = "CBECS"
    cbecs2018["PUBCLIM"] = cbecs_climate_zone
    cbecs2018_for_union = cbecs2018[["data_source", "EUI_kBTUft2"]]

    # ComStock Data Handling:
    #   Filter just to climate zone
    #   add in literal column for data_source for union and display in visualization
    #   Select data needed for union and visualization
    #   Rename columns for union
    #   Note if needed: in.comstock_building_type is building type for comstock data;
    comstock_cz = df_all[df_all["in.ashrae_iecc_climate_zone_2006"].str.contains(comstock_climate_zone)]
    comstock_cz["data_source"] = "ComStock"
    comstock_for_union = comstock_cz[["data_source", "EUI_kBTUft2"]]

    # ESPM Data Handling
    #   Filter just to climate zone
    espm_data_cz = espm_data[espm_data["ClimateZone"].str.contains(espm_climate_zone)]

    # Union CBECs and ComStock data for visualization
    union_df = pd.concat([cbecs2018_for_union, comstock_for_union], ignore_index=True)

    # Create box plot for unioned data
    sns.boxplot(data=union_df, x="EUI_kBTUft2", y="data_source")

    # layer in box plot for ESPM data (data in very different format so this step is required to get it to display correctly)
    sns.boxplot(
        data=[
            espm_data_cz["FifthPercentile"].mean(),
            espm_data_cz["TwentyFifthPercentile"].mean(),
            espm_data_cz["Median"].mean(),
            espm_data_cz["SeventyFifthPercentile"].mean(),
            espm_data_cz["NinetyFifthPercentile"].mean(),
        ],
        orient="h",
        color="tab:blue",
    )

    # Create percentile lines, averaged for all building types in zone
    std100_25th_percentile_mean = std100_25th_percentile[std_climate_zone].mean()
    std100_40th_percentile_mean = std100_40th_percentile[std_climate_zone].mean()
    std100_50th_percentile_mean = std100_50th_percentile[std_climate_zone].mean()

    plt.axvline(std100_25th_percentile_mean, color="gold", linestyle="--", linewidth=2, label="std100 25th Percentile")
    plt.axvline(std100_40th_percentile_mean, color="goldenrod", linestyle="--", linewidth=2, label="std100 40th Percentile")
    plt.axvline(std100_50th_percentile_mean, color="darkgoldenrod", linestyle="--", linewidth=2, label="std100 50th Percentile")

    plt.legend(bbox_to_anchor=(1.0, 1), loc="upper left")
    plt.xlabel("Site Energy EUI (kBtu/ft2)")
    plt.ylabel("Data Source")

    plt.title(f"Comparison of Climate Zone {espm_climate_zone} {climate_zone_descriptor} Standards for All Building Types", wrap=True)
    plt.xlim(0, 250)

    # Create label for ESPM data
    text_kwargs = {"ha": "right", "va": "baseline", "fontsize": 10, "color": "black"}
    plt.text(-4, 2.04, "ESPM", **text_kwargs)

    # Create footnote key for ESPM data
    text_kwargs = {"ha": "right", "va": "baseline", "fontsize": 10, "color": "black"}
    plt.text(-2.25, -0.01, "*", **text_kwargs)

    # Create note about climate zones in CBECs data
    text_kwargs = {"ha": "left", "va": "bottom", "fontsize": 10, "color": "black", "wrap": True}
    plt.text(255, 2, cbecs_footnote_text, **text_kwargs)

    # Save figure
    plt.subplots_adjust(right=0.7)
    plt.savefig(f"{figures_path}/all_buildings_cz{espm_climate_zone}.png", bbox_inches="tight")

In [ ]:
def retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
):
    # CBECS Data Handling:
    #   Filter to retail and office building types
    #       Note: Only included "2 - Office" and "25 - Retail other than mall"; did not include "23 - Strip shopping center" or "24 - Enclosed mall"
    #   calculate the total kbtu/ft2 for each building
    #   add in literal column for data_source for union
    #   Filter to climate zone (See lookups.py or climate_zone_crosswalk.xlsx in the cbecs data folder for additional definitions for different numbers, not the same as climate zone numbers)
    #   Select data needed for union and visualization
    cbecs2018_office_and_retail = cbecs2018[cbecs2018["PBA"].isin([2, 25])]
    cbecs2018_office_and_retail["EUI_kBTUft2"] = cbecs2018_office_and_retail["MFBTU"] / cbecs2018_office_and_retail["SQFT"]
    cbecs2018_office_and_retail["data_source"] = "CBECS"
    cbecs2018_office_and_retail["PUBCLIM"] = cbecs_climate_zone
    cbecs2018_office_and_retail_for_union = cbecs2018_office_and_retail[["data_source", "EUI_kBTUft2"]]

    # Comstock Data Handling:
    #   convert ComStock KWh to KBtu
    #   Filter to retail and office building types
    #       Note: did not include RetailStripMall
    #   Filter to climate zone
    #   add in literal column for data_source for union and display in visualization
    #   Select data needed for union and visualization
    #   Rename columns for union
    comstock_office_and_retail = df_all[
        df_all["in.comstock_building_type"].isin(["LargeOffice", "SmallOffice", "RetailStandalone", "MediumOffice"])
    ]
    comstock_office_and_retail_cz = comstock_office_and_retail[
        comstock_office_and_retail["in.ashrae_iecc_climate_zone_2006"].str.contains(comstock_climate_zone)
    ]
    comstock_office_and_retail_cz["data_source"] = "ComStock"
    comstock_office_and_retail_cz_for_union = comstock_office_and_retail_cz[["data_source", "EUI_kBTUft2"]]

    # ESPM Data Handling
    #   Filter to retail and office building types
    #       Note: did not include "Mailing Center/Post Office", "Medical Office", or "Veterinary Office"
    #   Filter to climate zone
    espm_data_office_and_retail = espm_data[
        espm_data["PropertyTypeSubcategory"].isin(["Other - Retail/Mall", "Retail Store", "Financial Office", "Office"])
    ]
    espm_data_office_and_retail = espm_data_office_and_retail[espm_data_office_and_retail["ClimateZone"].str.contains(espm_climate_zone)]

    # Union CBECs and ComStock data for visualization
    union_df_office_and_retail = pd.concat(
        [cbecs2018_office_and_retail_for_union, comstock_office_and_retail_cz_for_union],
        ignore_index=True,
    )

    # Create box plot for unioned data
    sns.boxplot(data=union_df_office_and_retail, x="EUI_kBTUft2", y="data_source")

    # layer in box plot for ESPM data (data in very different format so this step is required to get it to display correctly)
    sns.boxplot(
        data=[
            espm_data_office_and_retail["FifthPercentile"].mean(),
            espm_data_office_and_retail["TwentyFifthPercentile"].mean(),
            espm_data_office_and_retail["Median"].mean(),
            espm_data_office_and_retail["SeventyFifthPercentile"].mean(),
            espm_data_office_and_retail["NinetyFifthPercentile"].mean(),
        ],
        orient="h",
        color="tab:blue",
    )

    # Create percentile lines, averaged for all building types in zone
    # Filter to retail and office building types
    #   Note: Did not include "Medical office (nondiagnostic)", "Medical office (diagnostic)","Post office/postal center", "Courthouse/probation office","Enclosed mall", or "Strip shopping mall"
    std100_25th_percentile_office_and_retail = std100_25th_percentile[
        std100_25th_percentile["Commercial Building Type"].isin(
            ["Other retail", "Retail store", "Admin/professional office", "Government office", "Mixed-use office", "Other office"]
        )
    ]
    std100_25th_percentile_mean_office_and_retail = std100_25th_percentile_office_and_retail[std_climate_zone].mean()

    std100_40th_percentile_office_and_retail = std100_40th_percentile[
        std100_40th_percentile["Commercial Building Type"].isin(
            ["Other retail", "Retail store", "Admin/professional office", "Government office", "Mixed-use office", "Other office"]
        )
    ]
    std100_40th_percentile_mean_office_and_retail = std100_40th_percentile_office_and_retail[std_climate_zone].mean()

    std100_50th_percentile_office_and_retail = std100_50th_percentile[
        std100_50th_percentile["Commercial Building Type"].isin(
            ["Other retail", "Retail store", "Admin/professional office", "Government office", "Mixed-use office", "Other office"]
        )
    ]
    std100_50th_percentile_mean_office_and_retail = std100_50th_percentile_office_and_retail[std_climate_zone].mean()

    plt.axvline(std100_25th_percentile_mean_office_and_retail, color="gold", linestyle="--", label="std100 25th Percentile")
    plt.axvline(std100_40th_percentile_mean_office_and_retail, color="goldenrod", linestyle="--", label="std100 40th Percentile")
    plt.axvline(std100_50th_percentile_mean_office_and_retail, color="darkgoldenrod", linestyle="--", label="std100 50th Percentile")
    plt.legend(bbox_to_anchor=(1.0, 1), loc="upper left")
    plt.xlabel("Site Energy EUI (kBtu/ft2)")
    plt.ylabel("Data Source")
    plt.title(
        f"Comparison of Climate Zone {espm_climate_zone} {climate_zone_descriptor} Standards for Retail and Office Building Types",
        wrap=True,
    )
    plt.xlim(0, 250)

    # Create label for ESPM data
    text_kwargs = {"ha": "right", "va": "baseline", "fontsize": 10, "color": "black"}
    plt.text(-4, 2.04, "ESPM", **text_kwargs)

    # Create footnote key for ESPM data
    text_kwargs = {"ha": "right", "va": "baseline", "fontsize": 10, "color": "black"}
    plt.text(-2.25, -0.01, "*", **text_kwargs)

    # Create note about climate zones in CBECs data
    text_kwargs = {"ha": "left", "va": "bottom", "fontsize": 10, "color": "black", "wrap": True}
    plt.text(255, 2, cbecs_footnote_text, **text_kwargs)

    # Save figure
    plt.subplots_adjust(right=0.7)
    plt.savefig(f"{figures_path}/office_and_retail_buildings_cz{espm_climate_zone}.png", bbox_inches="tight")

In [ ]:
def all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
):
    # CBECS Data Handling:
    #   calculate the total kbtu/ft2 for each building
    #   add in literal column for data_source for union and display in visualization
    #   Filter to climate zone (See lookups.py or climate_zone_crosswalk.xlsx in the cbecs data folder for additional definitions for different numbers, not the same as climate zone numbers)
    #   Select data needed for union and visualization
    #   Note if needed: PBA is building type
    cbecs2018["EUI_kBTUft2"] = cbecs2018["MFBTU"] / cbecs2018["SQFT"]
    cbecs2018["data_source"] = "CBECS"
    cbecs2018["PUBCLIM"] = cbecs_climate_zone
    cbecs2018_for_union = cbecs2018[["data_source", "EUI_kBTUft2"]]

    # Comstock Data Handling:
    #   convert ComStock KWh to KBtu
    #   Filter just to climate zone
    #   add in literal column for data_source for union and display in visualization
    #   Select data needed for union and visualization
    #   Rename columns for union
    #   Note if needed: in.comstock_building_type is building type for comstock data;
    df_all["EUI_kBTUft2"] = df_all["out.site_energy.total.energy_consumption_intensity"] * 3.412
    comstock_cz = df_all[df_all["in.ashrae_iecc_climate_zone_2006"].str.contains(comstock_climate_zone)]
    comstock_cz["data_source"] = "ComStock"
    comstock_for_union = comstock_cz[["data_source", "EUI_kBTUft2"]]

    # ESPM Data Handling
    #   Filter just to climate zone
    espm_data_cz = espm_data[espm_data["ClimateZone"].str.contains(espm_climate_zone)]

    # -----------------------------------------------------------------
    # std100
    # -----------------------------------------------------------------
    std100_25th_percentile["percentile"] = "std100_Percentile_25"
    std100_25th_percentile_means = std100_25th_percentile.groupby("percentile")[std_climate_zone].mean().reset_index()
    std100_25th_percentile_means.index = ["std100_Percentile_25"]
    std100_25th_percentile_means = std100_25th_percentile_means.rename(columns={std_climate_zone: "EUI_kBTUft2"})
    std100_25th_percentile_means = std100_25th_percentile_means.drop(columns=["percentile"])

    std100_40th_percentile["percentile"] = "std100_Percentile_40"
    std100_40th_percentile_means = std100_40th_percentile.groupby("percentile")[std_climate_zone].mean().reset_index()
    std100_40th_percentile_means.index = ["std100_Percentile_40"]
    std100_40th_percentile_means = std100_40th_percentile_means.rename(columns={std_climate_zone: "EUI_kBTUft2"})
    std100_40th_percentile_means = std100_40th_percentile_means.drop(columns=["percentile"])

    std100_50th_percentile["percentile"] = "std100_Percentile_50"
    std100_50th_percentile_means = std100_50th_percentile.groupby("percentile")[std_climate_zone].mean().reset_index()
    std100_50th_percentile_means.index = ["std100_Percentile_50"]
    std100_50th_percentile_means = std100_50th_percentile_means.rename(columns={std_climate_zone: "EUI_kBTUft2"})
    std100_50th_percentile_means = std100_50th_percentile_means.drop(columns=["percentile"])

    # -----------------------------------------------------------------
    # ESPM - Note the ESPM data export does not have a 40th percentile
    # -----------------------------------------------------------------

    espm_data_cz_25th_percentile_means = espm_data_cz.groupby("ClimateZone")["TwentyFifthPercentile"].mean().reset_index()
    espm_data_cz_25th_percentile_means.index = ["ESPM_Percentile_25"]
    espm_data_cz_25th_percentile_means = espm_data_cz_25th_percentile_means.rename(columns={"TwentyFifthPercentile": "EUI_kBTUft2"})
    espm_data_cz_25th_percentile_means = espm_data_cz_25th_percentile_means.drop(columns=["ClimateZone"])

    espm_data_cz_50th_percentile_means = espm_data_cz.groupby("ClimateZone")["Median"].mean().reset_index()
    espm_data_cz_50th_percentile_means.index = ["ESPM_Percentile_50"]
    espm_data_cz_50th_percentile_means = espm_data_cz_50th_percentile_means.rename(columns={"Median": "EUI_kBTUft2"})
    espm_data_cz_50th_percentile_means = espm_data_cz_50th_percentile_means.drop(columns=["ClimateZone"])

    display(espm_data_cz_25th_percentile_means)
    # -----------------------------------------------------------------
    # CBECS
    # -----------------------------------------------------------------

    percentiles_cbecs = cbecs2018_for_union["EUI_kBTUft2"].quantile([0.25, 0.4, 0.5])
    percentiles_cbecs = percentiles_cbecs.T
    percentiles_cbecs.index = ["CBECS_Percentile_25", "CBECS_Percentile_40", "CBECS_Percentile_50"]

    # -----------------------------------------------------------------
    # ComStock
    # -----------------------------------------------------------------

    percentiles_comstock = comstock_for_union["EUI_kBTUft2"].quantile([0.25, 0.4, 0.5])
    percentiles_comstock = percentiles_comstock.T
    percentiles_comstock.index = ["ComStock_Percentile_25", "ComStock_Percentile_40", "ComStock_Percentile_50"]

    # -----------------------------------------------------------------
    # Data merge steps
    # -----------------------------------------------------------------

    percentiles_union = pd.concat(
        [
            percentiles_cbecs,
            percentiles_comstock,
            std100_25th_percentile_means,
            std100_40th_percentile_means,
            std100_50th_percentile_means,
            espm_data_cz_25th_percentile_means,
            espm_data_cz_50th_percentile_means,
        ],
        ignore_index=False,
    )

    percentiles_union = percentiles_union.reset_index()

    percentiles_union = percentiles_union.rename(columns={"index": "PercentileType"})

    # -----------------------------------------------------------------
    # Calculate percent differences, 25th percentiles
    # -----------------------------------------------------------------
    percentile_comparison_25 = percentiles_union[
        percentiles_union.PercentileType.isin(
            ["CBECS_Percentile_25", "ComStock_Percentile_25", "std100_Percentile_25", "ESPM_Percentile_25"]
        )
    ]

    var = percentile_comparison_25["EUI_kBTUft2"].to_numpy()
    arr = ((var - var[:, None]) / var) * -100
    percentile_comparison_25_matrix = pd.DataFrame(
        arr, columns=percentile_comparison_25["PercentileType"], index=percentile_comparison_25["PercentileType"]
    )

    # -----------------------------------------------------------------
    # Calculate percent differences, 50th percentiles
    # -----------------------------------------------------------------
    percentile_comparison_50 = percentiles_union[
        percentiles_union.PercentileType.isin(
            ["CBECS_Percentile_50", "ComStock_Percentile_50", "std100_Percentile_50", "ESPM_Percentile_50"]
        )
    ]

    var = percentile_comparison_50["EUI_kBTUft2"].to_numpy()
    arr = ((var - var[:, None]) / var) * -100
    percentile_comparison_50_matrix = pd.DataFrame(
        arr, columns=percentile_comparison_50["PercentileType"], index=percentile_comparison_50["PercentileType"]
    )

    # -----------------------------------------------------------------
    # Define a function for highlighting cells
    # -----------------------------------------------------------------

    def highlight_cells(val):
        color = (
            "#F1959B"
            if 20 < val < 40
            else "#F07470"
            if 40 < val < 60
            else "#EA4C46"
            if 60 < val < 80
            else "#DC1C13"
            if val > 80
            else "#F1959B"
            if -20 > val > -40
            else "#F07470"
            if -40 > val > -60
            else "#EA4C46"
            if -60 > val > -80
            else "#DC1C13"
            if val < -80
            else ""
        )
        return f"background-color: {color}"

    percentile_comparison_25_matrix = (
        percentile_comparison_25_matrix.style.applymap(highlight_cells)
        .format("{:.2f}")
        .set_caption(f"Comparison of Climate Zone {espm_climate_zone} {climate_zone_descriptor} Percentiles as Percent Difference")
    )
    percentile_comparison_50_matrix = (
        percentile_comparison_50_matrix.style.applymap(highlight_cells)
        .format("{:.2f}")
        .set_caption(f"Comparison of Climate Zone {espm_climate_zone} {climate_zone_descriptor} Percentiles as Percent Difference")
    )

    display(percentile_comparison_25)
    display(percentile_comparison_25_matrix)
    display(percentile_comparison_50)
    display(percentile_comparison_50_matrix)

# Climate Zone 1A

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 5
comstock_climate_zone = "1A"
espm_climate_zone = "1A"
std_climate_zone = "1A"
climate_zone_descriptor = "(Hot-Humid)"
cbecs_footnote_text = "*CBECs data uses a grouping of Hot or Very Hot climate zones (1A, 1B, 2A, 2B)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 2A

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 5
comstock_climate_zone = "2A"
espm_climate_zone = "2A"
std_climate_zone = "2A"
climate_zone_descriptor = "(Hot-Humid)"
cbecs_footnote_text = "*CBECs data uses a grouping of Hot or Very Hot climate zones (1A, 1B, 2A, 2B)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 2B

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 5
comstock_climate_zone = "2B"
espm_climate_zone = "2B"
std_climate_zone = "2B"
climate_zone_descriptor = "(Hot-Dry / Hot-Humid)"
cbecs_footnote_text = "*CBECs data uses a grouping of Hot or Very Hot climate zones (1A, 1B, 2A, 2B)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
# Uncertain why this is failing since it is the same as everything else, commenting out for now, may need help investigating
# ----------------------------------------------------------------------------------------------------------------------------
# all_buildings_percentile_comparisons(
#     cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
# )

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 3A

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 4
comstock_climate_zone = "3A"
espm_climate_zone = "3A"
std_climate_zone = "3A"
climate_zone_descriptor = "(Hot-Humid / Mixed Humid)"
cbecs_footnote_text = "*CBECs data uses a grouping of Warm climate zones (3A, 3B, 3C)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
# Uncertain why this is failing since it is the same as everything else, commenting out for now, may need help investigating
# ----------------------------------------------------------------------------------------------------------------------------
# all_buildings_percentile_comparisons(
#     cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
# )

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 3B

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 4
comstock_climate_zone = "3B"
espm_climate_zone = "3B"
std_climate_zone = "3B Other"
climate_zone_descriptor = "(Hot-Humid)"
cbecs_footnote_text = "*CBECs data uses a grouping of Warm climate zones (3A, 3B, 3C)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 3C

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 4
comstock_climate_zone = "3C"
espm_climate_zone = "3C"
std_climate_zone = "3C"
climate_zone_descriptor = "(Marine)"
cbecs_footnote_text = "*CBECs data uses a grouping of Warm climate zones (3A, 3B, 3C)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 4A

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 3
comstock_climate_zone = "4A"
espm_climate_zone = "4A"
std_climate_zone = "4A"
climate_zone_descriptor = "(Mixed-Humid)"
cbecs_footnote_text = "*CBECs data uses a grouping of Mixed Mild climate zones (4A, 4B, 4C)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 4B

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 3
comstock_climate_zone = "4B"
espm_climate_zone = "4B"
std_climate_zone = "4B"
climate_zone_descriptor = "(Mixed-Dry)"
cbecs_footnote_text = "*CBECs data uses a grouping of Mixed Mild climate zones (4A, 4B, 4C)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 4C

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 3
comstock_climate_zone = "4C"
espm_climate_zone = "4C"
std_climate_zone = "4C"
climate_zone_descriptor = "(Marine)"
cbecs_footnote_text = "*CBECs data uses a grouping of Mixed Mild climate zones (4A, 4B, 4C)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 5A

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 2
comstock_climate_zone = "5A"
espm_climate_zone = "5A"
std_climate_zone = "5A"
climate_zone_descriptor = "(Cold)"
cbecs_footnote_text = "*CBECs data uses a grouping of Cool climate zones (5A, 5B)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 5B

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 2
comstock_climate_zone = "5B"
espm_climate_zone = "5B"
std_climate_zone = "5B"
climate_zone_descriptor = "(Cold)"
cbecs_footnote_text = "*CBECs data uses a grouping of Cool climate zones (5A, 5B)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 6A

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 1
comstock_climate_zone = "6A"
espm_climate_zone = "6A"
std_climate_zone = "6A"
climate_zone_descriptor = "(Cold)"
cbecs_footnote_text = "*CBECs data uses a grouping of Cold and Very Cold climate zones (6A, 6B, 7, 8)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 6B

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 1
comstock_climate_zone = "6B"
espm_climate_zone = "6B"
std_climate_zone = "6B"
climate_zone_descriptor = "(Cold)"
cbecs_footnote_text = "*CBECs data uses a grouping of Cold and Very Cold climate zones (6A, 6B, 7, 8)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

# Climate Zone 7

In [ ]:
# Define parameters for functions

cbecs_climate_zone = 1
comstock_climate_zone = "7"
espm_climate_zone = "7"
std_climate_zone = 7
climate_zone_descriptor = "(Very Cold)"
cbecs_footnote_text = "*CBECs data uses a grouping of Cold and Very Cold climate zones (6A, 6B, 7, 8)"

In [ ]:
all_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)

In [ ]:
all_buildings_percentile_comparisons(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor
)

In [ ]:
retail_and_office_buildings_figure(
    cbecs_climate_zone, comstock_climate_zone, espm_climate_zone, std_climate_zone, climate_zone_descriptor, cbecs_footnote_text
)